### Notebook to clean up ethograms to be added to ground truth

In [1]:
import pandas as pd
from ipywidgets import HBox, VBox, Select, Button, Layout, RadioButtons
from fastplotlib import ImageWidget, Plot
from mesmerize_core.arrays import LazyVideo
from pathlib import Path
import numpy as np
from fastplotlib.graphics.selectors import LinearRegionSelector, LinearSelector

2023-06-16 09:32:46.465548: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-16 09:32:46.957934: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
ETHOGRAM_COLORS = {
    "lift": "b",
    "handopen": "green",
    "grab": "r",
    "sup": "cyan",
    "atmouth": "magenta",
    "chew": "yellow"
}

### dataframe to store cleaned ethograms

In [3]:
clean_df = pd.read_hdf('/data/caitlin/cleaned_ethograms.hdf')
clean_df

,trial,final_ethogram
0,M232_20170306_v074,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,M232_20170306_v024,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,M232_20170306_v025,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,M232_20170306_v059,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,M232_20170308_v028,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...
151,M259_20180402_v022,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
152,M275_20180830_v002,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
153,M278_20180921_v004,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
154,M282_20181203_v073,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


### load dataframe that has merged ethograms to go through and clean

In [4]:
all_potential_df = pd.read_hdf("/data/caitlin/exactly4_hand_labels.hdf")
all_potential_df

,mat_path,hand_labels,jaaba_labels,merged_ethogram
0,M232_20170307,"{'M232_20170307_v062': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170307_v062': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170307_v062': [[0, 0, 0, 0, 0, 0, 0, ..."
1,M232_20170308,"{'M232_20170308_v006': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170308_v006': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170308_v006': [[0, 0, 0, 0, 0, 0, 0, ..."
2,M234_20170329,"{'M234_20170329_v001': [[0, 0, 0, 0, 0, 0, 0, ...","{'M234_20170329_v001': [[0, 0, 0, 0, 0, 0, 0, ...","{'M234_20170329_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
3,M234_20170330,"{'M234_20170330_v044': [[0, 0, 0, 0, 0, 0, 0, ...","{'M234_20170330_v044': [[0, 0, 0, 0, 0, 0, 0, ...","{'M234_20170330_v044': [[0, 0, 0, 0, 0, 0, 0, ..."
4,M234_20170331,"{'M234_20170331_v002': [[0, 0, 0, 0, 0, 0, 0, ...","{'M234_20170331_v002': [[0, 0, 0, 0, 0, 0, 0, ...","{'M234_20170331_v002': [[0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
163,M336_20210612,"{'M336_20210612_v006': [[0, 0, 0, 0, 0, 0, 0, ...","{'M336_20210612_v006': [[0, 0, 0, 0, 0, 0, 0, ...","{'M336_20210612_v006': [[0, 0, 0, 0, 0, 0, 0, ..."
164,M336_20210613,"{'M336_20210613_v036': [[0, 0, 0, 0, 0, 0, 0, ...","{'M336_20210613_v036': [[0, 0, 0, 0, 0, 0, 0, ...",{}
165,M236_20170817,"{'M236_20170817_v002': [[0, 0, 0, 0, 0, 0, 0, ...","{'M236_20170817_v002': [[0, 0, 0, 0, 0, 0, 0, ...","{'M236_20170817_v002': [[0, 0, 0, 0, 0, 0, 0, ..."
166,M236_20170818,"{'M236_20170818_v042': [[0, 0, 0, 0, 0, 0, 0, ...","{'M236_20170818_v042': [[0, 0, 0, 0, 0, 0, 0, ...","{'M236_20170818_v042': [[0, 0, 0, 0, 0, 0, 0, ..."


In [5]:
data_path = Path('/data/caitlin/potential_ground_truth4/')

In [6]:
options = [k.stem for k in sorted(data_path.glob('*.avi'))]

In [7]:
len(options)

201

### get original merged ethograms that need to be cleaned

In [8]:
ethograms = dict()

In [9]:
for row in all_potential_df.iterrows():
    keys = row[1]['merged_ethogram'].keys()
    for k in keys:
        ethograms[k] = row[1]['merged_ethogram'][k]

In [10]:
data_path = Path('/data/caitlin/potential_ground_truth4/')

In [11]:
def trial_change(obj):
    selected_vid = data_path.joinpath(trial_selector.value).with_suffix('.avi')
    iw.set_data([LazyVideo(selected_vid)], reset_vmin_vmax=True)
    plot.clear()
    make_plot(plot)
    plot.auto_scale()

In [12]:
def clean_ethogram(obj):
    # get indices of selected region
    selected_ixs = plot.selectors[1].get_selected_indices(plot.graphics[0])
    # map behavior button value to index
    behavior_ix = behavior_buttons.options.index(behavior_buttons.value)
    # set indices of selected region to 
    if fill_values.value == "0":
        plot.graphics[behavior_ix].colors[selected_ixs[0]:selected_ixs[-1]] = "black"
    else:
        plot.graphics[behavior_ix].colors[selected_ixs[0]:selected_ixs[-1]] = ETHOGRAM_COLORS[behavior_buttons.value]

In [13]:
def save_new_ethogram(obj):
    # create new ethogram based off of indices that are not black
    trial_length = ethograms[trial_selector.value][0].shape[0]
    new_ethogram = np.zeros(shape=(6, trial_length))
    for i, graphic in enumerate(plot.graphics):
        non_zero_ixs = np.where(plot.graphics[i].colors[:] != np.array([0, 0, 0, 1]))[0]
        new_ethogram[i][non_zero_ixs] = 1
    # check if key in clean_df
    if trial_selector.value in trial_selector.value in list(clean_df["trial"]):
        ix = clean_df.loc[clean_df["trial"] == trial_selector.value].index[0]
        clean_df.loc[:,'final_ethogram'].loc[ix] = new_ethogram
    # else add to end of df
    else:
         clean_df.loc[len(clean_df.index)] = [trial_selector.value, new_ethogram]

    # save clean_df to disk
    clean_df.to_hdf('/data/caitlin/cleaned_ethograms.hdf', key='df')

In [14]:
def reset_ethogram(obj):
    old_ethogram = ethograms[trial_selector.value]
    for i, graphic in enumerate(plot.graphics):
        graphic.colors[old_ethogram[i] == 1] = list(ETHOGRAM_COLORS.values())[i]

In [15]:
def save_original_ethogram(obj):
    if trial_selector.value in trial_selector.value in list(clean_df["trial"]):
        ix = clean_df.loc[clean_df["trial"] == trial_selector.value].index[0]
        clean_df.loc[:,'final_ethogram'].loc[ix] = ethograms[trial_selector.value]
    # else add to end of df
    else:
         clean_df.loc[len(clean_df.index)] = [trial_selector.value, ethograms[trial_selector.value]]
    # save clean_df to disk
    clean_df.to_hdf('/data/caitlin/cleaned_ethograms.hdf', key='df')

In [16]:
# radio buttons to click which behavior needs to be changed 
behavior_buttons = RadioButtons(options=["lift", "handopen", "grab", "sup", "atmouth", "chew"], layout=Layout(width='auto'))
# radio button to check whether the value should be changed to zero or 1
fill_values = RadioButtons(options=["0", "1"], layout=Layout(width='auto'))
# button to clean the ethogram based on the values of the radio buttons
clean_button = Button(value=False, disabled=False, icon='broom',
                                       layout=Layout(width='auto'), tooltip='clean ethogram')
clean_button.on_click(clean_ethogram)
# save button to save new ethogram to cleaned_df and save df to disk also change the trial in marked_df to "good"
save_button = Button(value=False, disabled=False, icon='save',
                                       layout=Layout(width='auto'), tooltip='save clean ethogram')
save_button.on_click(save_new_ethogram)
    # check if key exists, and overwrite, otherwise add
# reset button to reset the ethogram to what is stored in marked_df and set key of marked_df to "clean"
reset_button = Button(value=False, disabled=False, icon='history',
                                       layout=Layout(width='auto'), tooltip='reset ethogram')
reset_button.on_click(reset_ethogram)
check_button = Button(value=False, disabled=False, icon='check',
                                       layout=Layout(width='auto'), tooltip='save original etho')
check_button.on_click(save_original_ethogram)

In [17]:
radio_box = HBox([behavior_buttons, fill_values])
clean_options = HBox([reset_button, clean_button, save_button, check_button])

In [18]:
trial_selector = Select(options=options)
trial_selector.observe(trial_change, "value")

In [19]:
iw = ImageWidget(data=[LazyVideo(data_path.joinpath(trial_selector.value).with_suffix('.avi'))])

RFBOutputContext()

/home/clewis7/repos/mesmerize-core/mesmerize_core/arrays/_video.py:87: UserWarning: min not implemented for LazyTiff, returning min of 0th index
  warn("min not implemented for LazyTiff, returning min of 0th index")
/home/clewis7/repos/mesmerize-core/mesmerize_core/arrays/_video.py:92: UserWarning: max not implemented for LazyTiff, returning min of 0th index
  warn("max not implemented for LazyTiff, returning min of 0th index")


In [20]:
def ethogram_event_handler(ev):
    ix = ev.pick_info["selected_index"]
    iw.sliders["t"].value = ix

In [21]:
plot = Plot(size=(500, 100))
def make_plot(plot):
        
    ethogram_array = ethograms[trial_selector.value]
    y_bottom = 0
    for i, b in enumerate(ETHOGRAM_COLORS.keys()):
        xs = np.arange(ethogram_array.shape[1], dtype=np.float32)
        ys = np.zeros(xs.size, dtype=np.float32)

        lg = plot.add_line(
            data=np.column_stack([xs, ys]),
            thickness=10,
            name=b
        )

        lg.colors = 0
        lg.colors[ethogram_array[i] == 1] = ETHOGRAM_COLORS[b]

        y_pos = (i * -10) - 1
        lg.position_y = y_pos

    ethogram_region_selector = LinearRegionSelector(
                bounds=(0, 50),
                limits=(0, ethogram_array.shape[1]),
                axis="x",
                origin=(0,-25),
                fill_color=(0, 0, 0, 0),
                parent=lg,
                size=(55),
            )
    
    ethogram_selector = LinearSelector(
                selection=0,
                limits=(0, ethogram_array.shape[1]),
                axis="x",
                parent=lg,
                end_points=(y_bottom, y_pos),
            )

    plot.add_graphic(ethogram_selector)
    plot.add_graphic(ethogram_region_selector)
    ethogram_selector.selection.add_event_handler(ethogram_event_handler)

RFBOutputContext()

In [22]:
make_plot(plot)
VBox([
    HBox([iw.show(), 
          VBox([
              trial_selector
              ])
            ]),
    HBox([plot.show(),
         VBox([radio_box, clean_options])
         ])
])

In [23]:
clean_df.iloc[-1]["trial"]

'M312_20200330_v005'

In [24]:
pd.read_hdf('/data/caitlin/cleaned_ethograms.hdf')

,trial,final_ethogram
0,M232_20170306_v074,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,M232_20170306_v024,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,M232_20170306_v025,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,M232_20170306_v059,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,M232_20170308_v028,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...
151,M259_20180402_v022,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
152,M275_20180830_v002,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
153,M278_20180921_v004,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
154,M282_20181203_v073,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [25]:
len(options)

201